<img src="https://github.com/hernancontigiani/ceia_memorias_especializacion/raw/master/Figures/logoFIUBA.jpg" width="500" align="center">


# Procesamiento de lenguaje natural
## LSTM Bot QA

### Datos
El objecto es utilizar datos disponibles del challenge ConvAI2 (Conversational Intelligence Challenge 2) de conversaciones en inglés. Se construirá un BOT para responder a preguntas del usuario (QA).\
[LINK](http://convai.io/data/)

In [1]:
!pip install --upgrade --no-cache-dir gdown --quiet

In [2]:
import re

import numpy as np
import pandas as pd

import tensorflow as tf
from keras.preprocessing.text import one_hot
from tensorflow.keras.utils import pad_sequences
from keras.models import Sequential
from keras.layers import Activation, Dropout, Dense
from keras.layers import Flatten, LSTM, SimpleRNN
from keras.models import Model
from tensorflow.keras.layers import Embedding
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer
from keras.layers import Input

In [3]:
# Descargar la carpeta de dataset
import os
import gdown
if os.access('data_volunteers.json', os.F_OK) is False:
    url = 'https://drive.google.com/uc?id=1awUxYwImF84MIT5-jCaYAPe2QwSgS1hN&export=download'
    output = 'data_volunteers.json'
    gdown.download(url, output, quiet=False)
else:
    print("El dataset ya se encuentra descargado")

Downloading...
From: https://drive.google.com/uc?id=1awUxYwImF84MIT5-jCaYAPe2QwSgS1hN&export=download
To: /content/data_volunteers.json
100%|██████████| 2.58M/2.58M [00:00<00:00, 144MB/s]


In [4]:
# dataset_file
import json

text_file = "data_volunteers.json"
with open(text_file) as f:
    data = json.load(f) # la variable data será un diccionario



In [5]:
# Observar los campos disponibles en cada linea del dataset
data[0].keys()

dict_keys(['dialog', 'start_time', 'end_time', 'bot_profile', 'user_profile', 'eval_score', 'profile_match', 'participant1_id', 'participant2_id'])

In [6]:
chat_in = []
chat_out = []

input_sentences = []
output_sentences = []
output_sentences_inputs = []
max_len = 30

def clean_text(txt):
    txt = txt.lower()
    txt.replace("\'d", " had")
    txt.replace("\'s", " is")
    txt.replace("\'m", " am")
    txt.replace("don't", "do not")
    txt = re.sub(r'\W+', ' ', txt)

    return txt

for line in data:
    for i in range(len(line['dialog'])-1):
        # vamos separando el texto en "preguntas" (chat_in)
        # y "respuestas" (chat_out)
        chat_in = clean_text(line['dialog'][i]['text'])
        chat_out = clean_text(line['dialog'][i+1]['text'])

        if len(chat_in) >= max_len or len(chat_out) >= max_len:
            continue

        input_sentence, output = chat_in, chat_out

        # output sentence (decoder_output) tiene <eos>
        output_sentence = output + ' <eos>'
        # output sentence input (decoder_input) tiene <sos>
        output_sentence_input = '<sos> ' + output

        input_sentences.append(input_sentence)
        output_sentences.append(output_sentence)
        output_sentences_inputs.append(output_sentence_input)

print("Cantidad de rows utilizadas:", len(input_sentences))

Cantidad de rows utilizadas: 6033


In [7]:
input_sentences[1], output_sentences[1], output_sentences_inputs[1]

('hi how are you ', 'not bad and you  <eos>', '<sos> not bad and you ')

### 2 - Preprocesamiento
Realizar el preprocesamiento necesario para obtener:
- word2idx_inputs, max_input_len
- word2idx_outputs, max_out_len, num_words_output
- encoder_input_sequences, decoder_output_sequences, decoder_targets

In [36]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

# Tokenización
input_tokenizer = Tokenizer()
input_tokenizer.fit_on_texts(input_sentences)
input_sequences = input_tokenizer.texts_to_sequences(input_sentences)
word2idx_inputs = input_tokenizer.word_index

output_tokenizer = Tokenizer(filters='')
output_tokenizer.fit_on_texts(output_sentences)
output_sequences = output_tokenizer.texts_to_sequences(output_sentences)
word2idx_outputs = output_tokenizer.word_index

# Determinación de la Longitud Máxima de Secuencia
max_input_len = max(len(s) for s in input_sequences)
max_out_len = max(len(s) for s in output_sequences)

# Número de palabras únicas en el output
num_words_output = len(word2idx_outputs) + 1

# Preparación de Secuencias de Encoder y Decoder
encoder_input_sequences = pad_sequences(input_sequences, maxlen=max_input_len)
decoder_output_sequences = pad_sequences(output_sequences, maxlen=max_out_len, padding='post')

# Preparación de targets para el decoder
decoder_targets = np.zeros((len(input_sentences), max_out_len, num_words_output), dtype='float32')
for i, seq in enumerate(decoder_output_sequences):
    for j, word_idx in enumerate(seq):
        if word_idx > 0:
            decoder_targets[i, j, word_idx] = 1.


### 3 - Preparar los embeddings
Utilizar los embeddings de Glove o FastText para transformar los tokens de entrada en vectores

In [12]:
import os
import gdown
if os.access('gloveembedding.pkl', os.F_OK) is False:
    url = 'https://drive.google.com/uc?id=1KY6avD5I1eI2dxQzMkR3WExwKwRq2g94&export=download'
    output = 'gloveembedding.pkl'
    gdown.download(url, output, quiet=False)
else:
    print("Los embeddings gloveembedding.pkl ya están descargados")

Downloading...
From (uriginal): https://drive.google.com/uc?id=1KY6avD5I1eI2dxQzMkR3WExwKwRq2g94&export=download
From (redirected): https://drive.google.com/uc?id=1KY6avD5I1eI2dxQzMkR3WExwKwRq2g94&export=download&confirm=t&uuid=d4c19f4a-61e5-48ed-8859-9e02d9bbaa38
To: /content/gloveembedding.pkl
100%|██████████| 525M/525M [00:04<00:00, 120MB/s]


In [13]:
# Cargar los embeddings de Glove
import pickle
import numpy as np
from keras.models import Model
from keras.layers import Input, Embedding

# Carga los embeddings de Glove
with open('gloveembedding.pkl', 'rb') as f:
    embeddings_index = pickle.load(f)

In [41]:
import logging
import os
from pathlib import Path
from io import StringIO
import pickle

class WordsEmbeddings(object):
    logger = logging.getLogger(__name__)

    def __init__(self):
        # load the embeddings
        words_embedding_pkl = Path(self.PKL_PATH)
        if not words_embedding_pkl.is_file():
            words_embedding_txt = Path(self.WORD_TO_VEC_MODEL_TXT_PATH)
            assert words_embedding_txt.is_file(), 'Words embedding not available'
            embeddings = self.convert_model_to_pickle()
        else:
            embeddings = self.load_model_from_pickle()
        self.embeddings = embeddings
        # build the vocabulary hashmap
        index = np.arange(self.embeddings.shape[0])
        # Dicctionarios para traducir de embedding a IDX de la palabra
        self.word2idx = dict(zip(self.embeddings['word'], index))
        self.idx2word = dict(zip(index, self.embeddings['word']))

    def get_words_embeddings(self, words):
        words_idxs = self.words2idxs(words)
        return self.embeddings[words_idxs]['embedding']

    def words2idxs(self, words):
        return np.array([self.word2idx.get(word, -1) for word in words])

    def idxs2words(self, idxs):
        return np.array([self.idx2word.get(idx, '-1') for idx in idxs])

    def load_model_from_pickle(self):
        self.logger.debug(
            'loading words embeddings from pickle {}'.format(
                self.PKL_PATH
            )
        )
        max_bytes = 2**28 - 1 # 256MB
        bytes_in = bytearray(0)
        input_size = os.path.getsize(self.PKL_PATH)
        with open(self.PKL_PATH, 'rb') as f_in:
            for _ in range(0, input_size, max_bytes):
                bytes_in += f_in.read(max_bytes)
        embeddings = pickle.loads(bytes_in)
        self.logger.debug('words embeddings loaded')
        return embeddings

    def convert_model_to_pickle(self):
        # create a numpy strctured array:
        # word     embedding
        # U50      np.float32[]
        # word_1   a, b, c
        # word_2   d, e, f
        # ...
        # word_n   g, h, i
        self.logger.debug(
            'converting and loading words embeddings from text file {}'.format(
                self.WORD_TO_VEC_MODEL_TXT_PATH
            )
        )
        structure = [('word', np.dtype('U' + str(self.WORD_MAX_SIZE))),
                     ('embedding', np.float32, (self.N_FEATURES,))]
        structure = np.dtype(structure)
        # load numpy array from disk using a generator
        with open(self.WORD_TO_VEC_MODEL_TXT_PATH, encoding="utf8") as words_embeddings_txt:
            embeddings_gen = (
                (line.split()[0], line.split()[1:]) for line in words_embeddings_txt
                if len(line.split()[1:]) == self.N_FEATURES
            )
            embeddings = np.fromiter(embeddings_gen, structure)
        # add a null embedding
        null_embedding = np.array(
            [('null_embedding', np.zeros((self.N_FEATURES,), dtype=np.float32))],
            dtype=structure
        )
        embeddings = np.concatenate([embeddings, null_embedding])
        # dump numpy array to disk using pickle
        max_bytes = 2**28 - 1 # # 256MB
        bytes_out = pickle.dumps(embeddings, protocol=pickle.HIGHEST_PROTOCOL)
        with open(self.PKL_PATH, 'wb') as f_out:
            for idx in range(0, len(bytes_out), max_bytes):
                f_out.write(bytes_out[idx:idx+max_bytes])
        self.logger.debug('words embeddings loaded')
        return embeddings

class GloveEmbeddings(WordsEmbeddings):
    WORD_TO_VEC_MODEL_TXT_PATH = 'glove.twitter.27B.50d.txt'
    PKL_PATH = 'gloveembedding.pkl'
    N_FEATURES = 50
    WORD_MAX_SIZE = max_input_len

class FasttextEmbeddings(WordsEmbeddings):
    WORD_TO_VEC_MODEL_TXT_PATH = 'cc.en.300.vec'
    PKL_PATH = 'fasttext.pkl'
    N_FEATURES = 300
    WORD_MAX_SIZE = 60

In [42]:
model_embeddings = GloveEmbeddings()

In [43]:
# Crear la Embedding matrix de las secuencias
print('preparing embedding matrix...')
MAX_VOCAB_SIZE = 8000
embed_dim = model_embeddings.N_FEATURES
words_not_found = []

# word_index provieen del tokenizer

nb_words = min(MAX_VOCAB_SIZE, len(word2idx_inputs)) # vocab_size
embedding_matrix = np.zeros((nb_words, embed_dim))
for word, i in word2idx_inputs.items():
    if i >= nb_words:
        continue
    embedding_vector = model_embeddings.get_words_embeddings(word)[0]
    if (embedding_vector is not None) and len(embedding_vector) > 0:

        embedding_matrix[i] = embedding_vector
    else:
        # words not found in embedding index will be all-zeros.
        words_not_found.append(word)

print('number of null word embeddings:', np.sum(np.sum(embedding_matrix**2, axis=1) == 0))

preparing embedding matrix...
number of null word embeddings: 38


### 4 - Entrenar el modelo
Entrenar un modelo basado en el esquema encoder-decoder utilizando los datos generados en los puntos anteriores. Utilce como referencias los ejemplos vistos en clase.

In [44]:
max_input_len

9

In [45]:
from keras.models import Model
from keras.layers import Input, LSTM, Dense

n_units = 128
encoder_inputs = Input(shape=(max_input_len))
encoder_embedding_layer = Embedding(
          input_dim=nb_words,
          output_dim=embed_dim,
          input_length=max_input_len,
          weights=[embedding_matrix],
          trainable=False)

encoder_inputs_x = encoder_embedding_layer(encoder_inputs)
encoder = LSTM(n_units, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs_x)
encoder_states = [state_h, state_c]
decoder_inputs = Input(shape=(max_out_len))
decoder_embedding_layer = Embedding(input_dim=num_words_output, output_dim=n_units, input_length=max_out_len)
decoder_inputs_x = decoder_embedding_layer(decoder_inputs)
decoder_lstm = LSTM(n_units, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs_x, initial_state=encoder_states)
decoder_dense = Dense(num_words_output, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
model.compile(loss='categorical_crossentropy', optimizer="Adam", metrics=['accuracy'])
model.summary()

Model: "model_4"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_9 (InputLayer)        [(None, 9)]                  0         []                            
                                                                                                  
 input_10 (InputLayer)       [(None, 10)]                 0         []                            
                                                                                                  
 embedding_5 (Embedding)     (None, 9, 50)                89950     ['input_9[0][0]']             
                                                                                                  
 embedding_6 (Embedding)     (None, 10, 128)              231168    ['input_10[0][0]']            
                                                                                            

In [46]:
# Encoder Model
encoder_model = Model(encoder_inputs, encoder_states)

# Decoder Model
decoder_state_input_h = Input(shape=(n_units,))
decoder_state_input_c = Input(shape=(n_units,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_inputs_single = Input(shape=(1,))
decoder_inputs_single_x = decoder_embedding_layer(decoder_inputs_single)
decoder_outputs, state_h, state_c = decoder_lstm(decoder_inputs_single_x, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model([decoder_inputs_single] + decoder_states_inputs, [decoder_outputs] + decoder_states)

In [47]:
# Entrenamiento
hist = model.fit(
    [encoder_input_sequences, decoder_output_sequences],  # Inputs del modelo
    decoder_targets,  # Targets
    epochs=15,
    validation_split=0.2
)

Epoch 1/15
151/151 [==============================] - 20s 63ms/step - loss: 2.4511 - accuracy: 0.1178 - val_loss: 2.2827 - val_accuracy: 0.1289
Epoch 2/15
151/151 [==============================] - 3s 19ms/step - loss: 2.1756 - accuracy: 0.1429 - val_loss: 2.1833 - val_accuracy: 0.1656
Epoch 3/15
151/151 [==============================] - 2s 15ms/step - loss: 2.0463 - accuracy: 0.1786 - val_loss: 2.0784 - val_accuracy: 0.1709
Epoch 4/15
151/151 [==============================] - 2s 14ms/step - loss: 1.9407 - accuracy: 0.1866 - val_loss: 2.0031 - val_accuracy: 0.1738
Epoch 5/15
151/151 [==============================] - 3s 17ms/step - loss: 1.8598 - accuracy: 0.1961 - val_loss: 1.9385 - val_accuracy: 0.1942
Epoch 6/15
151/151 [==============================] - 3s 18ms/step - loss: 1.7741 - accuracy: 0.2133 - val_loss: 1.8399 - val_accuracy: 0.2064
Epoch 7/15
151/151 [==============================] - 3s 17ms/step - loss: 1.6728 - accuracy: 0.2240 - val_loss: 1.7634 - val_accuracy: 0.219

### 5 - Inferencia
Experimentar el funcionamiento de su modelo. Recuerde que debe realizar la inferencia de los modelos por separado de encoder y decoder.

In [49]:
output_tokenizer = Tokenizer(filters='', oov_token='<sos>')
idx2word_outputs = {v: k for k, v in word2idx_outputs.items()}
if '<sos>' not in idx2word_outputs.values():
    idx2word_outputs[len(idx2word_outputs) + 1] = '<sos>'
if '<sos>' not in word2idx_outputs:
    word2idx_outputs['<sos>'] = len(word2idx_outputs) + 1

In [54]:
def respond(input_text):
    # Preprocesamiento de la entrada
    input_text = clean_text(input_text)
    input_seq = input_tokenizer.texts_to_sequences([input_text])
    input_seq = pad_sequences(input_seq, maxlen=max_input_len)

    # Encodear la secuencia de entrada
    states_value = encoder_model.predict(input_seq)

    # Crear una secuencia de longitud 1 para el token <sos>
    target_seq = np.zeros((1, 1))
    target_seq[0, 0] = word2idx_outputs['<sos>']

    # Recoger la respuesta
    output_sentence = []

    for _ in range(max_out_len):
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        # Elegir el siguiente token
        idx = np.argmax(output_tokens[0, 0, :])

        # Si es <eos>, terminar la generación
        if idx == word2idx_outputs['<eos>']:
            break

        # Añadir el token a la respuesta
        if idx > 0:
            output_sentence.append(idx2word_outputs[idx])

        # Actualizar la secuencia de entrada del decoder y los estados
        target_seq[0, 0] = idx
        states_value = [h, c]

    return ' '.join(output_sentence)

# Crear un diccionario inverso para el output
idx2word_outputs = {v: k for k, v in word2idx_outputs.items()}

# Usar la función respond para generar una respuesta
input_text = "How are you?"
print(respond(input_text))


1/1 [==============================] - 0s 29ms/step
and and and and and and and and and and
